In [1]:
import pandas as pd

In [43]:
df = pd.read_csv("../../dataset/nou_hearings_full_text.csv")

df2 = df["actor"].unique()
pd.DataFrame(df2)

,0
0,Abelia
1,Advokatforeningen
2,Agder fylkeskommune
3,Akademikerne
4,Alliansen ny landbrukspolitikk
...,...
205,WWF Verdens naturfond
206,Yara
207,Yrkesorganisasjonenes Sentralforbund (YS)
208,Z-Clix


In [44]:
df_arguments = pd.read_csv("../../dataset/util/argument_df.csv")

In [45]:
import spacy
from spacy.language import Language
from spacy_language_detection import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector(seed=1)

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [46]:
import re

# Loop through each row in the dataframe
cleaned_df = pd.DataFrame(columns=["paragraph","arguments"])
for i, paragraph, Response in df_arguments.itertuples():
    arguments = Response.split("\n")
    # Remove the first element of the list
    arguments = arguments[1:]
    # Remove the listing numbers
    
    cleaned_arguments = [re.sub(r"(\d+\.\ )|\*\ ", "", argument) for argument in arguments]

        # Remove if the argument is less than 15 characters
    cleaned_arguments = [argument for argument in cleaned_arguments if len(argument) > 15]
    
    # Remove empty strings
    #cleaned_arguments = [argument for argument in cleaned_arguments if argument]

    # Remove the sentences that does not exist in the paragraph
    for cleaned_argument in cleaned_arguments:
        if cleaned_argument.lower() not in paragraph.lower():
            cleaned_arguments.remove(cleaned_argument)

        # Check whether the cleaned argument is english then remove the argument
        doc = nlp(cleaned_argument)
        for j, sent in enumerate(doc.sents):
            if sent._.language["language"] == "en":
                #print("English", sent)
                try:
                    cleaned_arguments.remove(cleaned_argument)
                except:
                    pass


    #print(cleaned_arguments)
    cleaned_df.loc[i] = [paragraph, cleaned_arguments]




English Abelia has measured Norway's ability to transform towards a more sustainable economy over the past eight years, and the results show that Norway is heading in the wrong direction.
English Norway is doing well in renewable energy production, but little to save energy.
English The current structure of Norway's economy is not sustainable and requires a significant effort, including reprioritizing resources, acquiring new competencies, and paying greater attention to new growth industries, to achieve the necessary transformation.
English Including circular innovation as a central part of the strategy to reduce emissions is important.
English This approach is not only environmentally friendly, but it can also lead to increased economic efficiency and create new business opportunities.
English The elected representatives of the Norwegian Bar Association perform extensive voluntary and unpaid work to uphold the rule of law, legal certainty, and human rights.
English The hearing letter

In [60]:
# Add actor and actor label to the dataframe
df_actor_label = pd.read_csv("../../dataset/nou_hearings.csv")
df_actor_label = df_actor_label[["actor","actor_label","text"]]
# Merge the two dataframes
cleaned_df = cleaned_df.reset_index(drop=True)

# Change name of the column in df_actor_label to paragraph
df_actor_label.rename(columns={"text":"paragraph"}, inplace=True)

df_complete = pd.merge(cleaned_df, df_actor_label, on="paragraph")


In [4]:
# Remove rows that only contain [] in the arguments
df_complete = pd.read_csv("../../dataset/cleaned_arguments_in_context_learning.csv")
df_complete = df_complete[df_complete["text"].apply(lambda x: len(x) > 4)]
df_complete.drop(columns=["paragraph"], inplace=True)
df_complete

,text,actor,label
0,Abelia har 2900 medlemmer over hele landet. Ku...,Abelia,Interessegruppe
1,Utredningen gir et betydningsfullt perspektiv ...,Abelia,Interessegruppe
2,Klimapolitikken må favne bredere. Innovasjon e...,Abelia,Interessegruppe
3,"Norway had a good starting point in 2010, but ...",Abelia,Interessegruppe
4,Særlig fokus på følgende punkter i oppfølginge...,Abelia,Interessegruppe
...,...,...,...
3375,Å hente på bedre og mer samordnet areal- og tr...,ZERO,Frivillig organisasjon
3376,Digitale løsninger Samordning av logistikk og ...,ZERO,Frivillig organisasjon
3377,klimamålene kan bare innfris gjennom et aksele...,ZERO,Frivillig organisasjon
3378,Alle prosjekter som skal inn i NTP må kunne re...,ZERO,Frivillig organisasjon


In [5]:
df_complete.to_csv("../../dataset/cleaned_arguments_in_context_learning.csv", index=False)